In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pickle
path=r"C:\Users\galan\Chess_Prediction_Project\train dataset\combined_dataset.pkl"
with open(path, "rb") as file:
        X, y = pickle.load(file)

X

[array([[ 0. ,  0. ,  0. ,  0.6,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.2, -1. ],
        [ 0. ,  0. ,  0. ,  0. , -0.2,  0. ,  0. , -0.2],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,  0.2],
        [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  1. ,  0. ]]),
 array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ],
        [ 0.4, -0.2,  0. ,  0. ,  0. , -0.2, -0.2,  0. ],
        [ 0. ,  0. ,  0. ,  0. , -0.2,  0. ,  0. , -0.2],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ],
        [ 0.2,  0.2,  0. ,  0. ,  0. ,  0.2,  1. ,  0.2],
        [ 0. ,  0. , -0.6,  0. ,  0. ,  0. ,  0. ,  0. ]]),
 array([[-0.6,  0. ,  0. ,  0. , -0.6,  0. , -1. ,  0. ],
        [-

In [9]:
path=r"C:\Users\galan\Chess_Prediction_Project\train dataset\feature_dataset.pkl"
with open(path, "rb") as file:
        X_feature = pickle.load(file)

X_feature

array([[0. , 0. , 1. , ..., 0. , 0. , 0. ],
       [0. , 0. , 1. , ..., 0. , 0. , 0. ],
       [0. , 1. , 0. , ..., 0. , 0. , 1. ],
       ...,
       [1. , 0. , 0. , ..., 0. , 1.2, 0. ],
       [1. , 0. , 0. , ..., 0. , 4.7, 1. ],
       [0. , 1. , 0. , ..., 0. , 1.2, 0. ]])

In [10]:
len(X_feature)

21456

In [11]:
X = np.array(X)
X

array([[[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. , -0.2, -1. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. , -0.2],
        ...,
        [ 0. ,  0.2,  0. , ...,  0. ,  0.2,  0. ],
        [ 0. ,  0. ,  0. , ...,  0.2,  0. ,  0.2],
        [ 0. ,  0. ,  0. , ...,  0. ,  1. ,  0. ]],

       [[ 0. ,  0. ,  0. , ...,  0. , -1. ,  0. ],
        [ 0.4, -0.2,  0. , ..., -0.2, -0.2,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. , -0.2],
        ...,
        [ 0. ,  0. ,  0. , ...,  0. ,  0.2,  0. ],
        [ 0.2,  0.2,  0. , ...,  0.2,  1. ,  0.2],
        [ 0. ,  0. , -0.6, ...,  0. ,  0. ,  0. ]],

       [[-0.6,  0. ,  0. , ...,  0. , -1. ,  0. ],
        [-0.2, -0.2, -0.2, ..., -0.2, -0.2, -0.2],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        ...,
        [ 0.2,  0. ,  0.2, ...,  0. , -0.4,  0.2],
        [ 0. ,  0. ,  0. , ...,  0.2,  0.2,  0. ],
        [ 0.6,  0. ,  0.5, ...,  0.6,  1. ,  0. ]],

       ...,

       [[-0.6,  0. , -0.

In [16]:
X_feature.shape
X = np.array(X)
X_feature = np.array(X_feature)
y = np.array(y)

from sklearn.preprocessing import MinMaxScaler

# Assuming X_feature is a numpy array of shape (num_samples, 26)
scaler = MinMaxScaler()

# Fit the scaler on X_feature and transform it
X_feature = scaler.fit_transform(X_feature)

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.regularizers import l2

def build_model(board_input_shape=(8, 8, 1), phase_input_shape=(26,)):
    # Input for 8x8 chessboard (single channel)
    board_input = Input(shape=board_input_shape, name='board_input')

    # CNN layers for chessboard (8x8 matrix) with L2 regularization
    x1 = Conv2D(100, (3, 3), padding='same')(board_input)
    x1 = LeakyReLU(alpha=0.01)(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Dropout(0.25)(x1)
    
    x1 = Flatten()(x1)

    # Input for phase-specific features
    phase_input = Input(shape=phase_input_shape, name='phase_input')

    # Dense layers for phase-specific features with L2 regularization
    x2 = Dense(100, kernel_regularizer=l2(0.0001))(phase_input)
    x2 = LeakyReLU(alpha=0.01)(x2)
   
    # Concatenate the CNN output and the phase-specific features
    combined = Concatenate()([x1, x2])
    
    # Additional dense layers after concatenation with L2 regularization
    
    x =Dropout(0.5) (Dense(100,  kernel_regularizer=l2(0.0001))(combined))
    x = LeakyReLU(alpha=0.01)(x)
   
    # Output layer (3 classes: white, black, equal)
    output = Dense(3, activation='softmax')(x)

    # Create and compile the model
    model = Model(inputs=[board_input, phase_input], outputs=output)
    model.compile(optimizer = Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_model()

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ board_input (InputLayer)      │ (None, 8, 8, 1)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 8, 8, 32)          │             320 │ board_input[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 4, 4, 32)          │               0 │ conv2d_2[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 4, 4, 64)          │          18,496 │ max_pooling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 2, 2, 64)          │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ phase_input (InputLayer)      │ (None, 26)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 256)               │               0 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 100)               │           2,700 │ phase_input[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 356)               │               0 │ flatten_1[0][0],           │
│                               │                           │                 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 100)               │          35,700 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 3)                 │             303 │ dense_4[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 57,519 (224.68 KB)

 Trainable params: 57,519 (224.68 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets, specifying the test size as 2000
X_train, X_test, X_feature_train, X_feature_test, y_train, y_test = train_test_split(
    X, X_feature, y, test_size=4000, random_state=42
)

# Verify the split
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

Training set size: 17456 samples
Test set size: 4000 samples


In [19]:
from sklearn.model_selection import KFold
import numpy as np

# Assuming X_train, X_feature_train, y_train are numpy arrays
# Here, we will apply KFold to all datasets: X_train, X_feature_train, and y_train

kfold = KFold(n_splits=6, shuffle=True, random_state=42)

for train_idx, val_idx in kfold.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    X_feature_train_fold, X_feature_val_fold = X_feature_train[train_idx], X_feature_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Train the model using the training fold and validate using the validation fold
    model_trained = model.fit(
        [X_train_fold, X_feature_train_fold], y_train_fold,  # Train with both chessboard and phase features
        epochs=50,
        batch_size=64,
        validation_data=([X_val_fold, X_feature_val_fold], y_val_fold)  # Validate with both chessboard and phase features
    )

Epoch 1/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.4488 - loss: 6.3694 - val_accuracy: 0.5285 - val_loss: 1.0359
Epoch 2/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5227 - loss: 1.0251 - val_accuracy: 0.5629 - val_loss: 0.9791
Epoch 3/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5684 - loss: 0.9674 - val_accuracy: 0.5739 - val_loss: 0.9412
Epoch 4/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5885 - loss: 0.9354 - val_accuracy: 0.5790 - val_loss: 0.9316
Epoch 5/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5893 - loss: 0.9292 - val_accuracy: 0.5797 - val_loss: 0.9335
Epoch 6/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5922 - loss: 0.9167 - val_accuracy: 0.5859 - val_loss: 0.9197
Epoch 7/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.6056 - loss: 0.9022 - val_accuracy: 0.6079 - val_loss: 0.8819
Epoch 8/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.6126 - loss: 0.8885 - val_accur

In [20]:
model.evaluate([X_test,X_feature_test],
               y_test)

125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6248 - loss: 0.8374


[0.826461672782898, 0.6334999799728394]

In [21]:
X.shape

(21456, 8, 8)

In [22]:
import pickle
path='test dataset\\test_dataset.pkl'
with open(path, "rb") as file:
        Xtest, id = pickle.load(file)

In [23]:
len(Xtest)

8539

In [24]:
import pickle
path='test dataset\\feature_test_dataset.pkl'
with open(path, "rb") as file:
        Xtest_feature = pickle.load(file)

In [25]:
from sklearn.preprocessing import MinMaxScaler

# Assuming X_feature is a numpy array of shape (num_samples, 26)
scaler = MinMaxScaler()

# Fit the scaler on X_feature and transform it
Xtest_feature = scaler.fit_transform(Xtest_feature)

In [26]:
Xtest_feature

array([[0.        , 1.        , 0.        , ..., 0.        , 0.58762887,
        0.6       ],
       [0.        , 1.        , 0.        , ..., 0.42857143, 0.46391753,
        0.8       ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.54639175,
        0.6       ],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 0.68041237,
        1.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.42268041,
        0.6       ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.6185567 ,
        0.6       ]])

In [27]:
Xtest = np.array(Xtest)
Xtest_feature = np.array(Xtest_feature)
id = np.array(id)
y_predicted = model.predict([Xtest,Xtest_feature])

267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [28]:
len(y_predicted)
id

array([10037061, 10009951, 10101628, ...,  9982844,  9987342,  9988237])

In [29]:
y_predicted[0]

array([0.41864243, 0.5077359 , 0.07362165], dtype=float32)

In [30]:
df = pd.read_csv("C:\\Users\\galan\\Chess_Prediction_Project\\chess_test_csv.csv")
df

image_id  evaluation
0     26065374         NaN
1     32481096         NaN
2     36997677         NaN
3     34417938         NaN
4     28905181         NaN
...        ...         ...
8535  31917698         NaN
8536    393246         NaN
8537  26319610         NaN
8538  16378115         NaN
8539  30711130         NaN

[8540 rows x 2 columns]

In [31]:
df.rename(columns={'image_id': 'id'}, inplace=True)
df

id  evaluation
0     26065374         NaN
1     32481096         NaN
2     36997677         NaN
3     34417938         NaN
4     28905181         NaN
...        ...         ...
8535  31917698         NaN
8536    393246         NaN
8537  26319610         NaN
8538  16378115         NaN
8539  30711130         NaN

[8540 rows x 2 columns]

In [32]:
import numpy as np

y_predicted = np.argmax(y_predicted, axis=1)

In [33]:
len(y_predicted)

8539

In [34]:
for idx, val in enumerate(df.id):
    index=np.where(id==val)[0]
    if y_predicted[index] == 0:
        df['evaluation'][idx] = "equal"
    elif y_predicted[index] == 1:
        df['evaluation'][idx] = "white"
    else:
        df['evaluation'][idx] = "black"

In [35]:
df = df[~df['id'].duplicated(keep='first')]
df

id evaluation
0     26065374      equal
1     32481096      black
2     36997677      white
3     34417938      equal
4     28905181      black
...        ...        ...
8535  31917698      equal
8536    393246      equal
8537  26319610      black
8538  16378115      black
8539  30711130      black

[8539 rows x 2 columns]

In [36]:
df.to_csv("result.csv", index=False)

In [37]:
import pickle

# Suppose you have a trained model named 'model'
# For example, model = SomeClassifier().fit(X_train, y_train)

# Save the model to a file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
